<a href="https://colab.research.google.com/github/saralieber/CS_Studio/blob/master/Review_Ch4_MachineLearning_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vShZZ4EVYQQKWcdQ1JzXxIFj8PER0zk9LwF67YY1BR3iXkWBopwFrGS8-VL1bSACKCRBgOsnY3rAqHU/pub?output=csv'

In [3]:
letters_table = pd.read_csv(url)
letters_table.head()

,Survived,Name,Length,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,"Braund, Mr. Owen Harris",23,2,1,0,1,1,0,0,1,1,0,0,0,1,2,1,0,0,4,1,0,1,0,1,0,0,0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",51,2,2,2,1,4,1,3,2,2,1,0,2,2,3,2,0,0,5,3,1,1,0,0,0,2,0
2,1,"Heikkinen, Miss. Laina",22,2,0,0,0,2,0,0,1,4,0,2,1,1,3,0,0,0,0,2,0,0,0,0,0,0,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",44,3,0,1,0,6,1,0,2,1,1,0,5,2,0,0,1,1,2,2,2,2,0,0,0,2,0
4,0,"Allen, Mr. William Henry",24,2,0,0,0,2,0,0,1,2,0,0,4,2,2,0,0,0,2,0,0,0,0,1,0,1,0


In [4]:
# Machine Learning
## Set up Training and Testing Sets

# For this example, say you want 30% of the sample to be in the training set and 70% to be in the testing set
n_for_test_set = int(len(letters_table)/3) # using int() rounds the result to the nearest whole number
n_for_test_set # 297

n_for_train_set = len(letters_table) - n_for_test_set # the other 70% goes in the training set
n_for_train_set # 594

594

In [5]:
# Creating a Holdout Sample
## First, shuffle the rows in the table randomly
## Second, split the shuffled table into training & testing sets based on n's above

set_seed = 1234 # to be able to replicate random shuffling

import numpy as np
rsgen = np.random.RandomState(set_seed) # numpy's random number generator
print(rsgen)

# Use the .sample() method to randomly shuffle the rows of the original table
shuffled_table = letters_table.sample(frac=1, random_state = rsgen) # frac=1 means shuffle the entire table
shuffled_table.head()

RandomState(MT19937)


,Survived,Name,Length,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
523,1,"Hippach, Mrs. Louis Albert (Ida Sophia Fischer)",47,4,1,2,1,2,1,0,4,5,0,0,2,1,0,2,3,0,3,4,1,1,0,0,0,0,0
778,0,"Kilgannon, Mr. Thomas J",23,2,0,0,0,0,0,1,1,1,1,1,1,2,3,2,0,0,1,1,1,0,0,0,0,0,0
760,0,"Garfirth, Mr. John",18,1,0,0,0,0,1,1,2,1,1,0,0,1,1,1,0,0,3,0,1,0,0,0,0,0,0
496,1,"Eustis, Miss. Elizabeth Mussey",30,1,1,0,0,4,0,0,1,3,0,0,1,2,0,0,0,0,0,6,2,2,0,0,0,1,1
583,0,"Ross, Mr. John Hugo",19,0,0,0,0,0,0,1,2,0,1,0,0,1,1,3,0,0,2,2,0,1,0,0,0,0,0


In [0]:
# Split the randomly shuffled table into the training and testing set

testing_table = shuffled_table[:n_for_test_set]
testing_table = testing_table.reset_index(drop=True) # reset the indices 

training_table = shuffled_table[n_for_test_set:]
training_table = training_table.reset_index(drop=True)

In [7]:
## Convert the training set into a list containing all the rows of data (from column 3 to the end)

training_passengers_matrix = []

# Training Table
for i in range(len(training_table)):
  rows = training_table.iloc[i].tolist()
  sliced_rows = rows[2:]
  training_passengers_matrix.append(sliced_rows)

for i in range(5):
  print(training_passengers_matrix[i])

#
training_outcomes = training_table['Survived'].tolist()

[35, 1, 2, 1, 0, 2, 1, 0, 0, 2, 0, 0, 3, 2, 2, 4, 0, 0, 2, 4, 1, 2, 0, 0, 0, 0, 0]
[25, 1, 0, 0, 2, 2, 1, 0, 0, 0, 1, 0, 1, 1, 1, 2, 0, 0, 2, 3, 3, 0, 0, 0, 0, 0, 0]
[18, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 2, 1, 0, 0, 1, 2, 1, 1, 0, 0, 0, 0, 0]
[21, 2, 0, 0, 3, 2, 0, 0, 2, 0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0]
[22, 1, 0, 0, 0, 3, 0, 0, 0, 2, 1, 0, 2, 2, 1, 1, 0, 0, 2, 2, 0, 0, 0, 1, 0, 0, 0]


In [8]:
# Prepping for the ML part
!rm -r 'uo_puddles'
my_github_name = 'uo-puddles'
clone_url = f'https://github.com/{my_github_name}/uo_puddles.git'
!git clone $clone_url  
import uo_puddles.uo_puddles as up

Cloning into 'uo_puddles'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 225 (delta 133), reused 64 (delta 33), pack-reused 0
Receiving objects: 100% (225/225), 56.74 KiB | 691.00 KiB/s, done.
Resolving deltas: 100% (133/133), done.


In [9]:
# An example version using one row from the testing set

test_row_0 = testing_table.iloc[0].tolist()
test_values_0 = test_row_0[2:] # slice off columns 1 and 2
print(test_values_0)

# Now, use the ordered_distancex_matrix to get the equivlant of the sorted_passengers_matrix from Ch.3
## Which is the calculated euclidean distance between a reference (in this case, the one person from the test set) and everyone else in the training set, and sort the distances from lowest to highest
ordered_crowd = up.ordered_distances_matrix(test_values_0, training_passengers_matrix) # this is measuring similarity based on euclidean distances calculated from alphabet scores for each person's name
ordered_crowd[:5] # check out first 5


# Choose a value for K
k = 5

# Get the row indexes for the top five people in the ordered_crowd (the top five people most similar to the sample test participant)
expert_list = []

for i in range(k):
  pair = ordered_crowd[i] # [index, distance] - the index for the first five people in ordered_crowd & distance of each from the test participant
  index = pair[0] # the first item in pair (the index for each participant)
  expert_list.append(index) # store the indexes
expert_list # 51, 154, 491, 396, 24, by "expert" we mean the people most similar to the test subject


# Next, see how the outcome of the five people most similar to the test subject compare to the outcome of the test subject
expert_outcomes = []

for i in range(k):
  index = expert_list[i]
  outcome = training_outcomes[index] # this is the 'Survived' score for the 'expert' participants
  expert_outcomes.append(outcome)
expert_outcomes # 1, 1, 0, 1, 1 
# 4 people survived (1), 1 person perished (0)


# The above are saved as functions in uo_puddles
## training_passengers_matrix - a list of rows (each row is a list itself) representing the length and counts of the 26 letters of the alphabet
## traininglabels - the is the survived column from letters_table in list form

[47, 4, 1, 2, 1, 2, 1, 0, 4, 5, 0, 0, 2, 1, 0, 2, 3, 0, 3, 4, 1, 1, 0, 0, 0, 0, 0]


[1, 1, 0, 1, 1]

In [10]:
# Shortcut way of doing what we did in code above using the built-in functions from up
## Using the knn function from uo_puddles (up)

training_labels = letters_table['Survived'].tolist()

up.knn(test_values_0, training_passengers_matrix, training_labels, 5, 'euclidean')
# up.knn(reference_person, all_other_people, outcomes_for_all_other_people, value_of_k, similarity_measure)
## Our model predicts 1 for the test_row subject (because a majority of people similar to the test_row subject scored 1 on the outcome variable)

# Were we correct?
print(test_row_0) # Yes! This person scored 1 on 'Survived' (the first column value)

[1, 'Hippach, Mrs. Louis Albert (Ida Sophia Fischer)', 47, 4, 1, 2, 1, 2, 1, 0, 4, 5, 0, 0, 2, 1, 0, 2, 3, 0, 3, 4, 1, 1, 0, 0, 0, 0, 0]


In [0]:
# Cosine Similarity
## Another way of measuring similarity besides euclidean distance

<img src='https://www.dropbox.com/s/7rtuzw37hgl1oi8/ed_vs_cos.png?raw=1' height=300>

In [0]:
# Euclidean distance is d (the distance between two points)
# Cosine similarity is cosine(theta) - the cosine of the angle formed by vectors connecting the origin to each of the points

The formula for calculating cosine similarity:

<img src='https://www.dropbox.com/s/oi1ttx99hf0uejn/cosine.png?raw=1'>

In [14]:
# Calculating cosine similarity
## For this example, use the test_row subject as A, and use the first person from the training_passengers_matrix as B

A = test_values_0
B = training_passengers_matrix[0]


# Compute the numerator 

A_cross_B = []

for i in range(len(A)):
  A_times_B = A[i]*B[i]
  A_cross_B.append(A_times_B)

numerator = sum(A_cross_B)
numerator


# Computer the denominator

A_squared_list = []
B_squared_list = []

for i in range(len(A)):
  A_sqaured = (A[i])**2
  B_squared = (B[i])**2
  A_squared_list.append(A_sqaured)
  B_squared_list.append(B_squared)

denominator = ((sum(A_squared_list))**0.5)*((sum(B_squared_list))**0.5)
denominator


# Compute cosine 
cosine = numerator/denominator
cosine


# Interpreting Cosine
## Cosine gives a value between 0 and 1
## The smaller the cosine value, the more dissimlar the two vectors are
## The best value (most similar) is a cosine of 1; the worst value (most dissimilar) is a cosine of 0


# The above process has been written as a function in uo_puddles
## up.cosine_similarity(vector1, vector2)
up.cosine_similarity(A,B)

0.9844045418547783

In [18]:
# Chapter 4 Exercises

## See how well you can predict the outcome of the people in the testing table from using the training table as the "crowd" that you build your predictions based on
### For each row in the testing table, call up.knn to get a prediction using k = 5 and similarity measure 'cosine'
### Store the predictions in a new list

n = len(testing_table)

predictions = []

for i in range(n):
  rows_to_lists = testing_table.iloc[i].tolist()
  test_values = rows_to_lists[2:]
  preds = up.knn(test_values, training_passengers_matrix, training_labels, 5, 'cosine')
  predictions.append(preds)

print(predictions[:20])

[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
